# PsychoDNet training

In [29]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
import random
import time
import math
from tensorflow.python.eager import context
import pandas as pd
from tensorflow.keras.layers import Input
from sklearn.metrics import roc_auc_score
from sklearn import metrics
import pylab as plt
from tensorflow import keras

os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_USE_FRONTEND'] = '1'
# os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'
# print(f"Random seed set as {seed}")
os.environ["PYTHONHASHSEED"] = '0'
    
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
    
    
### set background seed
seed = 42
# setup_seed(seed)

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
# tf.random.uniform([1], seed=seed)

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

## import processed 2d matrix data for training

In [30]:
x_train = np.load('./data_intermediate/x_train.npy',allow_pickle=True)
y_train = np.load('./data_intermediate/y_train.npy',allow_pickle=True)
print(x_train[0].shape)
x_train = np.expand_dims(x_train, -1)
x_train = x_train.astype('float32') 


num_classes = 2
input_shape = x_train.shape[1:]

(7, 7)


In [31]:
print(x_train[0][0:2])
print(y_train[0][0:2])

[[[1.        ]
  [0.04338772]
  [0.17487054]
  [1.        ]
  [1.        ]
  [1.        ]
  [0.13080749]]

 [[0.82400584]
  [0.4432482 ]
  [0.8478585 ]
  [0.66367406]
  [0.7751856 ]
  [0.15940285]
  [0.11855265]]]
[1. 0.]


## Define architecture of PsychoDNet

In [32]:
# define bottleneck
class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, growth_rate, drop_rate):
        super(BottleNeck, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filters=4 * growth_rate,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=growth_rate,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.dropout = tf.keras.layers.Dropout(rate=drop_rate)
        
        self.listLayers = [self.bn1,
                           tf.keras.layers.Activation("relu"),
                           self.conv1,
                           self.bn2,
                           tf.keras.layers.Activation("relu"),
                           self.conv2,
                           self.dropout]

    def call(self, x):
        tf.random.set_seed(seed)
        y = x
        for layer in self.listLayers.layers:
            y = layer(y)
        y = tf.keras.layers.concatenate([x,y], axis=-1)
        return y
# define dense block
class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate, drop_rate=0.5):
        super(DenseBlock, self).__init__()
        self.num_layers = num_layers
        self.growth_rate = growth_rate
        self.drop_rate = drop_rate
        self.listLayers = []
        for _ in range(num_layers):
            self.listLayers.append(BottleNeck(growth_rate=self.growth_rate, drop_rate=self.drop_rate))

    def call(self, x):
        tf.random.set_seed(seed)
        for layer in self.listLayers.layers:
            x = layer(x)
        return x

# define transition
class TransitionLayer(tf.keras.layers.Layer):
    def __init__(self, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = tf.keras.layers.BatchNormalization()
        self.conv = tf.keras.layers.Conv2D(filters=out_channels,
                                           kernel_size=(1, 1),
                                           strides=1,
                                           padding="same")
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2, 2),
                                              strides=2,
                                              padding="same")

    def call(self, inputs):
        tf.random.set_seed(seed)
        x = self.bn(inputs)
        x = tf.keras.activations.relu(x)
        x = self.conv(x)
        x = self.pool(x)
        return x

# define dense net
class DenseNet(tf.keras.Model):
    def __init__(self, num_init_features, growth_rate, block_layers, compression_rate, drop_rate):
        super(DenseNet, self).__init__()

        self.conv = tf.keras.layers.Conv2D(filters=num_init_features,
                                           kernel_size=(3, 3),
                                           strides=1,
                                           input_shape = (7,7,1),
                                           padding="same")
        self.bn = tf.keras.layers.BatchNormalization()
        self.num_channels = num_init_features
        self.dense_block_1 = DenseBlock(num_layers=block_layers[0], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[0]
        self.num_channels = compression_rate * self.num_channels
        self.transition_1 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_2 = DenseBlock(num_layers=block_layers[1], growth_rate=growth_rate, drop_rate=drop_rate)
        self.num_channels += growth_rate * block_layers[1]
        self.num_channels = compression_rate * self.num_channels
        self.transition_2 = TransitionLayer(out_channels=int(self.num_channels))
        self.dense_block_3 = DenseBlock(num_layers=block_layers[2], growth_rate=growth_rate, drop_rate=drop_rate)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=2,
                                        activation=tf.keras.activations.softmax)

    def call(self, inputs):
        tf.random.set_seed(seed)
        x = self.conv(inputs)
        x = self.bn(x)
        x = tf.keras.activations.relu(x)

        x = self.dense_block_1(x)
        x = self.transition_1(x)
        x = self.dense_block_2(x)
        x = self.transition_2(x)
        x = self.dense_block_3(x)

        x = self.avgpool(x)
        x = self.fc(x)

        return x

In [33]:
if __name__ == '__main__':
    model = DenseNet(num_init_features=32, growth_rate=8, block_layers=[3,4,3], compression_rate=0.5, drop_rate=0.3)
    model.build(input_shape=(None, 7, 7, 1))
 
    # Adding this call to the call() method solves it all
    model.call(Input(shape=(7, 7, 1)))

model.summary()

Model: "dense_net_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_782 (Conv2D)          (None, 7, 7, 32)          320       
_________________________________________________________________
batch_normalization_782 (Bat (None, 7, 7, 32)          128       
_________________________________________________________________
dense_block_102 (DenseBlock) (None, 7, 7, 56)          11736     
_________________________________________________________________
transition_layer_68 (Transit (None, 4, 4, 28)          1820      
_________________________________________________________________
dense_block_103 (DenseBlock) (None, 4, 4, 60)          15648     
_________________________________________________________________
transition_layer_69 (Transit (None, 2, 2, 30)          2070      
_________________________________________________________________
dense_block_104 (DenseBlock) (None, 2, 2, 54)         

# training with 10-fold cv

In [34]:
from sklearn.model_selection import KFold
### define parameters
num_folds = 10
batch_size = 256
no_epochs = 200

np.random.seed(42)
shuffle_ix = np.random.permutation(np.arange(len(x_train)))
x_train = x_train[shuffle_ix]
y_train = y_train[shuffle_ix]
print(shuffle_ix[:5])
print(x_train[0][0:2])
print(y_train[0][0:2])

kfold = KFold(n_splits=num_folds, shuffle=True, random_state = seed)

[ 5575 20866 21599 17767 16200]
[[[0.10028079]
  [0.0486789 ]
  [0.19985205]
  [0.35672265]
  [0.16218597]
  [0.0603992 ]
  [0.22237274]]

 [[0.03977959]
  [0.69922537]
  [0.63637286]
  [0.2642445 ]
  [0.484491  ]
  [0.36677998]
  [0.21734653]]]
[1. 0.]


In [35]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

class ParameterTraining(object):
    def __init__(self, num_init_features, growth_rate, block_layers, compression_rate, drop_rate, no_epochs_1, no_epochs_2, lr_1, lr_2):
        self.num_init_features = num_init_features
        self.growth_rate = growth_rate
        self.block_layers = block_layers
        self.compression_rate = compression_rate
        self.drop_rate = drop_rate
        self.no_epochs_1 = no_epochs_1
        self.no_epochs_2 = no_epochs_2
        self.lr_1 = lr_1
        self.lr_2 = lr_2
        
    
    def train(self):
        result_fin = pd.DataFrame()
        for num_init_features in self.num_init_features:
            print('****************** num_init_features **************************--》', num_init_features)
            for growth_rate in self.growth_rate:
                print('****************** growth_rate **************************--》', growth_rate)
                for compression_rate in self.compression_rate:
                    print('****************** compression_rate **************************--》', compression_rate)
                    for drop_rate in self.drop_rate:
                        print('****************** drop_rate **************************--》', drop_rate)
                        start = time.time() ### estimate running time for a 10-fold cv of a group of parameters
                        val_acc_per_fold = []
                        val_loss_per_fold = []
                        val_prec_per_fold = []
                        val_rec_per_fold = []
                        val_spec_per_fold = []
                        val_mcc_per_fold = []
                        val_auc_per_fold = []
                        
                        fold_no = 1
                        
                        for train, val in kfold.split(x_train, y_train):

                            model = DenseNet(num_init_features=num_init_features, growth_rate=growth_rate, block_layers=block_layers, compression_rate=compression_rate, drop_rate=drop_rate)
                            model.compile(loss='categorical_crossentropy',
                                          optimizer=tf.keras.optimizers.Adam(lr=lr_1),
                                          # optimizer=tf.keras.optimizers.SGD(lr=lr,momentum=0.9),
                                          metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])
                            print('------------------------------------------------------------------------')
                            print('num_init_features %d , growth_rate %d , compression_rate %.2f , drop_rate %.2f' % (num_init_features,growth_rate,compression_rate,drop_rate))
                            print(f'Training for fold {fold_no} ...')
                            print('No. of negative/positive samples in training %d / %d' %  (np.sum(np.argmax(y_train[train], axis=1)==0),np.sum(np.argmax(y_train[train], axis=1)==1)))
                            print('No. of negative/positive samples in validation %d / %d' % (np.sum(np.argmax(y_train[val], axis=1)==0),np.sum(np.argmax(y_train[val], axis=1)==1)))
                            history = model.fit(x_train[train], y_train[train],
                                                batch_size=256,epochs=no_epochs_1,
                                                validation_data=(x_train[val], y_train[val]),
                                                verbose = 0, # verbose：日志显示, verbose = 0 为不在标准输出流输出日志信息, verbose = 1 为输出进度条记录, verbose = 2 为每个epoch输出一行记录
                                                callbacks = EarlyStop
                                               )
                            model.compile(loss='categorical_crossentropy',
                                          optimizer=tf.keras.optimizers.Adam(lr=lr_2),
                                          metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])
                            history = model.fit(x_train[train], y_train[train],
                                                batch_size=256,epochs=no_epochs_2,
                                                validation_data=(x_train[val], y_train[val]),
                                                verbose = 0, # verbose：日志显示, verbose = 0 为不在标准输出流输出日志信息, verbose = 1 为输出进度条记录, verbose = 2 为每个epoch输出一行记录
                                                callbacks = EarlyStop
                                               )
                            y_pred_val = model.predict(x_train[val])
                            y_pred_val = np.argmax(y_pred_val,axis=1)
                            y_val = np.argmax(y_train[val],axis=1)
                            val_prec_per_fold.append(metrics.precision_score(y_val, y_pred_val)) # precision
                            val_rec_per_fold.append(metrics.recall_score(y_val, y_pred_val)) # recall
                            val_mcc_per_fold.append(metrics.matthews_corrcoef(y_val, y_pred_val)) # mcc
                            cm = metrics.confusion_matrix(y_val,y_pred_val) # confusion matrix
                            val_spec_per_fold.append(cm[0,0]/(cm[0,0]+cm[0,1])) # specificity
                            
                            val_acc_per_fold.append(history.history['val_accuracy'][-1])
                            val_loss_per_fold.append(history.history['val_loss'][-1])
                            val_auc_per_fold.append(history.history['val_auc'][-1])
                            fold_no += 1
                         
                        print('time (%s)' % (timeSince(start)))
                        print('------------------------------------------------------------------------')
                        print('Average scores for all folds:')
                        print(f'> Val_Accuracy: %.4f (+- %.4f)' % (np.mean(val_acc_per_fold),np.std(val_acc_per_fold)))
                        print(f'> Val_Loss: %.4f (+- %.4f)' % (np.mean(val_loss_per_fold),np.std(val_loss_per_fold)))        
                        print('------------------------------------------------------------------------')
        return model,val_acc_per_fold,val_prec_per_fold,val_rec_per_fold,val_spec_per_fold,val_mcc_per_fold,val_auc_per_fold

# training with settled parameters

In [36]:
num_init_features = [32]
growth_rate = [8]
block_layers = [3,4,3]
compression_rate = [0.5]
drop_rate = [0.3]
no_epochs_1 = 10
no_epochs_2 = 200
lr_1 = 1e-3
lr_2 = 1e-4

EarlyStop = keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 20, min_delta = 0, mode = 'auto')

PT = ParameterTraining(num_init_features, growth_rate, block_layers, compression_rate, drop_rate, no_epochs_1, no_epochs_2, lr_1, lr_2)
# Densenet = PT.train()
PsychoDNet,acc,prec,rec,spec,mcc,auc = PT.train()

****************** num_init_features **************************--》 32
****************** growth_rate **************************--》 8
****************** compression_rate **************************--》 0.5
****************** drop_rate **************************--》 0.3
------------------------------------------------------------------------
num_init_features 32 , growth_rate 8 , compression_rate 0.50 , drop_rate 0.30
Training for fold 1 ...
No. of negative/positive samples in training 11734 / 8318
No. of negative/positive samples in validation 1275 / 953
------------------------------------------------------------------------
num_init_features 32 , growth_rate 8 , compression_rate 0.50 , drop_rate 0.30
Training for fold 2 ...
No. of negative/positive samples in training 11683 / 8369
No. of negative/positive samples in validation 1326 / 902
------------------------------------------------------------------------
num_init_features 32 , growth_rate 8 , compression_rate 0.50 , drop_rate 0.30
T

## model performance

In [37]:
print(f'> Val_Accuracy: %.4f (+- %.4f)' % (np.mean(acc),np.std(acc)))
print(f'> Val_Precision: %.4f (+- %.4f)' % (np.mean(prec),np.std(prec)))
print(f'> Val_Recall: %.4f (+- %.4f)' % (np.mean(rec),np.std(rec)))
print(f'> Val_Specificity: %.4f (+- %.4f)' % (np.mean(spec),np.std(spec)))
print(f'> Val_MCC: %.4f (+- %.4f)' % (np.mean(mcc),np.std(mcc)))
print(f'> Val_AUC: %.4f (+- %.4f)' % (np.mean(auc),np.std(auc)))

> Val_Accuracy: 0.9599 (+- 0.0046)
> Val_Precision: 0.9489 (+- 0.0159)
> Val_Recall: 0.9556 (+- 0.0106)
> Val_Specificity: 0.9630 (+- 0.0127)
> Val_MCC: 0.9178 (+- 0.0088)
> Val_AUC: 0.9915 (+- 0.0020)


### PsychoDNet prediction example

In [40]:
np.random.seed(42)
sample_ix = np.random.permutation(np.arange(100))
x_sample = x_train[sample_ix]
y_sample = y_train[sample_ix]
threshold = 0.0392

In [41]:
y_pred_sample = PsychoDNet.predict(x_sample)
print(f'> AUC: %.4f ' % (metrics.roc_auc_score(y_sample, y_pred_sample)))

> AUC: 1.0000 


In [43]:
y_pred_sample_binary = (y_pred_sample[:,1] >= threshold).astype(int)
y_sample_binary = np.argmax(y_sample,axis=1)
print(f'> Accuracy: %.3f ' % (metrics.accuracy_score(y_sample_binary, y_pred_sample_binary)))
print(f'> Precision: %.3f ' % (metrics.precision_score(y_sample_binary, y_pred_sample_binary)))
print(f'> Recall: %.3f ' % (metrics.recall_score(y_sample_binary, y_pred_sample_binary)))
cm = metrics.confusion_matrix(y_sample_binary,y_pred_sample_binary) # confusion matrix
print(f'> Specificity: %.3f ' % (cm[0,0]/(cm[0,0]+cm[0,1])))
print(f'> MCC: %.3f ' % (metrics.matthews_corrcoef(y_sample_binary, y_pred_sample_binary)))

> Accuracy: 1.000 
> Precision: 1.000 
> Recall: 1.000 
> Specificity: 1.000 
> MCC: 1.000 
